# [모듈 3.1] Code Pipeline 생성 하기

## 참고 자료
- https://docs.aws.amazon.com/codepipeline/latest/userguide/pipelines-create.html


# 1. 환경 설정


In [2]:
%store -r hello_repository_name
%store -r hello_code_build_project_name
%store -r account_id
%store -r region
%store -r bucket

# 2. 파라미터 설정
### 생성한 Code Pipeline Role 의 ARN 을 넣어 주세요. 아래를 지우고 넣습니다.
##### pipeline_role_name = "arn:aws:iam::XXXXXX:role/code-pipeline-gsmoon"


In [1]:
hello_code_pipeline_name = "Hello-Code-Pipeline"
# pipeline_role_name = "arn:aws:iam::XXXXXXX:role/code-pipeline-gsmoon"
pipeline_role_name = 'arn:aws:iam::057716757052:role/code-pipeline-gsmoon'
branch_name = 'master'


# 3. 파이프라인 정의를  JSON 정의

Code Pipeline 의 정의를 Json 으로 두개의 스테이지로 정의 합니다.
- (1) Source
    - 이전 노트북에서 정의한 Code Commit 리파지토리를 정의 합니다. (hello-code-repo)
- (2) Build
    - 이전 노트북에서 정의한 Code Build Project 정의 합니다. (Hello_Code_Build)    
    
여기서 정의한 Code Pipeline (Hello-Code-Pipeline) 은 위의 두개의 스테이지를 실행을 하게 됩니다.

In [4]:
pipeline_dic = {
    "pipeline": {
        "name": hello_code_pipeline_name,
        "roleArn": pipeline_role_name,
        "artifactStore": {
            "type": "S3",
            "location": bucket
        },
        "stages": [
            {
                "name": "Source",
                "actions": [
                    {
                        "name": "Source",
                        "actionTypeId": {
                            "category": "Source",
                            "owner": "AWS",
                            "provider": "CodeCommit",
                            "version": "1"
                        },
                        "runOrder": 1,
                        "configuration": {
                            "BranchName": branch_name,
                            "OutputArtifactFormat": "CODE_ZIP",
                            "PollForSourceChanges": "false",
                            "RepositoryName": hello_repository_name
                        },
                        "outputArtifacts": [
                            {
                                "name": "SourceArtifact"
                            }
                        ],
                        "inputArtifacts": [],
                        "region": region,
                        "namespace": "SourceVariables"
                    }
                ]
            },
            {
                "name": "Build",
                "actions": [
                    {
                        "name": "BuildAndExecuteSageMakerPipeline",
                        "actionTypeId": {
                            "category": "Build",
                            "owner": "AWS",
                            "provider": "CodeBuild",
                            "version": "1"
                        },
                        "runOrder": 1,
                        "configuration": {
                            "ProjectName": hello_code_build_project_name
                        },
                        "outputArtifacts": [
                            {
                                "name": "BuildArtifact"
                            }
                        ],
                        "inputArtifacts": [
                            {
                                "name": "SourceArtifact"
                            }
                        ],
                        "region": region,
                        "namespace": "BuildVariables"
                    }
                ]
            }
        ],
        "version": 1
    },
}

## 3.1. Dic 포맷을 JSON 으로 변경

In [5]:
import json

pipeline_json = json.dumps(pipeline_dic)
json.loads(pipeline_json)

{'pipeline': {'name': 'Hello-Code-Pipeline',
  'roleArn': 'arn:aws:iam::057716757052:role/code-pipeline-gsmoon',
  'artifactStore': {'type': 'S3',
   'location': 'sagemaker-us-east-1-057716757052'},
  'stages': [{'name': 'Source',
    'actions': [{'name': 'Source',
      'actionTypeId': {'category': 'Source',
       'owner': 'AWS',
       'provider': 'CodeCommit',
       'version': '1'},
      'runOrder': 1,
      'configuration': {'BranchName': 'master',
       'OutputArtifactFormat': 'CODE_ZIP',
       'PollForSourceChanges': 'false',
       'RepositoryName': 'hello-code-repo'},
      'outputArtifacts': [{'name': 'SourceArtifact'}],
      'inputArtifacts': [],
      'region': 'us-east-1',
      'namespace': 'SourceVariables'}]},
   {'name': 'Build',
    'actions': [{'name': 'BuildAndExecuteSageMakerPipeline',
      'actionTypeId': {'category': 'Build',
       'owner': 'AWS',
       'provider': 'CodeBuild',
       'version': '1'},
      'runOrder': 1,
      'configuration': {'ProjectN

## 3.2. JSON 파일 저장

In [6]:
json_file_path = 'src/hello_pipeline.json'
with open(json_file_path, "w") as outfile:
    outfile.write(pipeline_json)

In [7]:
with open(json_file_path) as json_file:
    json_data = json.load(json_file)
    
json_data    

{'pipeline': {'name': 'Hello-Code-Pipeline',
  'roleArn': 'arn:aws:iam::057716757052:role/code-pipeline-gsmoon',
  'artifactStore': {'type': 'S3',
   'location': 'sagemaker-us-east-1-057716757052'},
  'stages': [{'name': 'Source',
    'actions': [{'name': 'Source',
      'actionTypeId': {'category': 'Source',
       'owner': 'AWS',
       'provider': 'CodeCommit',
       'version': '1'},
      'runOrder': 1,
      'configuration': {'BranchName': 'master',
       'OutputArtifactFormat': 'CODE_ZIP',
       'PollForSourceChanges': 'false',
       'RepositoryName': 'hello-code-repo'},
      'outputArtifacts': [{'name': 'SourceArtifact'}],
      'inputArtifacts': [],
      'region': 'us-east-1',
      'namespace': 'SourceVariables'}]},
   {'name': 'Build',
    'actions': [{'name': 'BuildAndExecuteSageMakerPipeline',
      'actionTypeId': {'category': 'Build',
       'owner': 'AWS',
       'provider': 'CodeBuild',
       'version': '1'},
      'runOrder': 1,
      'configuration': {'ProjectN

# 4. 파이프라인 실행
- 파이프라인을 생성과 동시에 실행 합니다.

In [8]:
%%sh -s {json_file_path}
json_file_path=$1
echo $json_file_path
echo $PWD
aws codepipeline create-pipeline --cli-input-json file://$json_file_path
# aws codepipeline create-pipeline --cli-input-json file://src/hello_pipeline.json

src/hello_pipeline.json
/home/ec2-user/SageMaker/Neural-Collaborative-Filtering-On-SageMaker/3_MLOps/3_hello-codepipeline
{
    "pipeline": {
        "name": "Hello-Code-Pipeline",
        "roleArn": "arn:aws:iam::057716757052:role/code-pipeline-gsmoon",
        "artifactStore": {
            "type": "S3",
            "location": "sagemaker-us-east-1-057716757052"
        },
        "stages": [
            {
                "name": "Source",
                "actions": [
                    {
                        "name": "Source",
                        "actionTypeId": {
                            "category": "Source",
                            "owner": "AWS",
                            "provider": "CodeCommit",
                            "version": "1"
                        },
                        "runOrder": 1,
                        "configuration": {
                            "BranchName": "master",
                            "OutputArtifactFormat": "CODE_ZIP",
   

# 5. 파이프라인 실행 상태 확인

## 5.1. AWS CLI 로 확인

In [12]:
%%sh -s {hello_code_pipeline_name}
pipeline_name=$1
echo $pipeline_name
aws codepipeline get-pipeline-state --name $pipeline_name

Hello-Code-Pipeline
{
    "pipelineName": "Hello-Code-Pipeline",
    "pipelineVersion": 1,
    "stageStates": [
        {
            "stageName": "Source",
            "inboundTransitionState": {
                "enabled": true
            },
            "actionStates": [
                {
                    "actionName": "Source",
                    "currentRevision": {
                        "revisionId": "0e22caf0db28b3767368ac875592fef283a71b78"
                    },
                    "latestExecution": {
                        "actionExecutionId": "dd5301cd-968d-494b-9c8c-d4e0f7f53686",
                        "status": "Succeeded",
                        "summary": "Add new files\n",
                        "lastStatusChange": 1671259070.485,
                        "externalExecutionId": "0e22caf0db28b3767368ac875592fef283a71b78"
                    },
                    "entityUrl": "https://console.aws.amazon.com/codecommit/home?region=us-east-1#/repository/hello-cod

# 6. 변수 저장

In [13]:
%store hello_code_pipeline_name

Stored 'hello_code_pipeline_name' (str)


# 7. Code Pipelien 콘솔에서 확인

## 7.1. Code Pipeline 콘솔

![code_pipeline_console.png](img/code_pipeline_console.png)

## 7.2. Hello-Code-Pipeline 상세

![hello_code_pipeline_stages.png](img/hello_code_pipeline_stages.png)

## 7.3. Code Build Projectg 로그 및 결과 확인
- [중요] 실제 작업시 "에러" 가 발생하면 아래 화면의 "로그" 를 확인하면서 디버깅 하시면 됩니다.

![build_logs.png](img/build_logs.png)